In [1]:
import gradio as gr
from transformers import pipeline
import numpy as np
import openai
import warnings
import os
from dotenv import load_dotenv
load_dotenv()

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

warnings.filterwarnings('ignore')

C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [10]:
# Preference
# Initialize the ASR pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe_and_save(audio):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))
    transcription = transcriber({"sampling_rate": sr, "raw": y})["text"]

    # Write the transcription to a file
    with open("transcription_movie_selection.txt", "w") as file:
        file.write(transcription)
    
    return transcription


# Create the Gradio interface
demo = gr.Interface(
    transcribe_and_save,
    gr.Audio(sources=["microphone"], label="Tell me how your day was and what things you enjoy in movies!"),
    "text",
)

demo.launch(share=True)
#change to True for outside URL


Running on local URL:  http://127.0.0.1:7868
Running on public URL: https://5c624ba28bc4d87589.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\gradio\queueing.py", line 522, in process_events
    response = await route_utils.call_process_api(
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\gradio\route_utils.py", line 260, in call_process_api
    output = await app.get_blocks().process_api(
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\gradio\blocks.py", line 1689, in process_api
    result = await self.call_function(
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\gradio\blocks.py", line 1255, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\anyio\to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\anyio\_backends\_asyncio.py", line 818, in run_sync_in_worker_thread
    return awai

In [11]:
#pure movie selection analysis with OpenAI
#preprocess txt file

with open('transcription_movie_selection.txt', 'r', encoding="utf-8") as file:
    text = file.read()


In [12]:
# Function time

def get_preference(question):
    messages = [
        {"role": "system", "content": "Determine the emotions of the text as anger, disgust, fear, joy, neutral, sadness, or suprise and based on the provided text select three of the following genres: Action, Adventure, Animation, Comedy, Crime, Documentary, Drama, Family, Fantasy, History, Horror, Music, Mystery, Romance, Science Fiction, TV Movie, Thriller, War, and Western."},
        {"role": "user", "content": question}
    ]
    
    # Use the chat completions method
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=messages
    )
    
    # Extract the answer from the response
    answer = response.choices[0].message.content
    return answer

def analyze_document(filename):
    with open(filename, 'r') as file:
        text = file.read()
    preference = get_preference(text)
    print(f"{preference}")

def analyze_document(filename, movie_output):
    with open(filename, 'r') as file:
        text = file.read()
    preference = get_preference(text)
    
    # Write the output to a text file
    with open(movie_output, 'w') as outfile:
        outfile.write(f"{preference}\n")

analyze_document('transcription_movie_selection.txt', 'movie_output.txt')

with open('movie_output.txt') as file:
    lines = file.readlines()

# Display the lines
print(text)
for line in lines:
    print(line.strip())


 How Matt's Wood could a Woodcock Chock if Woodcock could chuck wood and I like movies with dogs cats chainsaws and dirt roads.
Emotions: Neutral
Genres: Comedy, Adventure, Action


In [6]:
#Change Gradio Interface (gradio theme builder)

gr.themes.builder()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----

To create a public link, set `share=True` in `launch()`.
